In [1]:
# load packages

import os
import pandas as pd
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data.descriptors.sets import SmilesDesc
from qsprpred.data.sampling.splits import GBMTRandomSplit, GBMTDataSplit, ManualSplit
from split import random_global_equilibrated_random_split, dissimilaritydrive_global_balanced_cluster_split
from qsprpred.extra.gpu.models.chemprop import ChempropModel
from qsprpred.models import CrossValAssessor, TestSetAssessor
from qsprpred.models.early_stopping import EarlyStoppingMode
from qsprpred.models import SklearnModel
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
# from qsprpred.plotting.regression import CorrelationPlot
from plotting_regression_MT import CorrelationPlot

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:
df=pd.read_csv('/home/s2861704/ppk/test_kin/output/TK/TK_split.csv', sep=',')
display(df)
print(df.columns)

,SMILES,O60674,P00519,P00533,P04626,P04629,P06213,P06239,P07333,P07949,...,Q06187,Q06418,Q07912,Q08345,Q08881,Q12866,Q16288,Q16620,Q16832,split
0,BrC(Cn1ncc2c(N3CCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
1,BrC(Cn1ncc2c(N3CCCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,BrC(Cn1ncc2c(NCc3ccccc3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
3,Brc1cc(CNc2ncc(Br)c(Nc3cc(C4CC4)[nH]n3)n2)on1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
4,Brc1cc2c(NCc3ccccc3)ncnc2s1,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36999,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37000,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37001,c1ncc(-c2cc(CNc3nc(Nc4cc(C5CC5)[nH]n4)cc(N4CCO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37002,c1ncc(CN2CCC(n3cnc4cnc5[nH]ccc5c43)CC2)cn1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


Index(['SMILES', 'O60674', 'P00519', 'P00533', 'P04626', 'P04629', 'P06213',
       'P06239', 'P07333', 'P07949', 'P08069', 'P08581', 'P09581', 'P09619',
       'P10721', 'P11362', 'P12931', 'P16234', 'P17948', 'P21802', 'P22455',
       'P22607', 'P23458', 'P29317', 'P29597', 'P30530', 'P35968', 'P36888',
       'P43405', 'P52333', 'P54760', 'Q02763', 'Q05397', 'Q06187', 'Q06418',
       'Q07912', 'Q08345', 'Q08881', 'Q12866', 'Q16288', 'Q16620', 'Q16832',
       'split'],
      dtype='object')


In [3]:
targets_kinase200 = df.columns[1:42].tolist()
display(targets_kinase200)

['O60674',
 'P00519',
 'P00533',
 'P04626',
 'P04629',
 'P06213',
 'P06239',
 'P07333',
 'P07949',
 'P08069',
 'P08581',
 'P09581',
 'P09619',
 'P10721',
 'P11362',
 'P12931',
 'P16234',
 'P17948',
 'P21802',
 'P22455',
 'P22607',
 'P23458',
 'P29317',
 'P29597',
 'P30530',
 'P35968',
 'P36888',
 'P43405',
 'P52333',
 'P54760',
 'Q02763',
 'Q05397',
 'Q06187',
 'Q06418',
 'Q07912',
 'Q08345',
 'Q08881',
 'Q12866',
 'Q16288',
 'Q16620',
 'Q16832']

In [ ]:
for target in targets_kinase200:
    print(target)
    df_st = df[["SMILES", target, "split"]]
    print(df_st)
    df_st_nonan = df_st.dropna(axis=0)
    
    print(df_st_nonan)

    # Create dataset
    os.makedirs("output/final/data", exist_ok=True)
    ds_name = "QSPRDataset_" + target
    dataset = QSPRDataset(
        df=df_st_nonan,
        store_dir="/home/s2861704/ppk/test_kin/output/final/data",
        name=ds_name,
        target_props=[{"name": target, "task": "REGRESSION"}],
        random_state=42,
        overwrite=True
    )
    dataset.getDF()

    # calculate compound features and split dataset into train and test
    feature_calculators = [MorganFP(radius=3, nBits=2048)]
    dataset.prepareDataset(
        split=ManualSplit(splitcol=dataset.df["split"], trainval="train", testval="test"),
        feature_calculators=feature_calculators,
        recalculate_features=True,
    )
    print(f"Number of samples in train set: {len(dataset.y)}")
    print(f"Number of samples in test set: {len(dataset.y_ind)}")

    os.makedirs("output/final/models", exist_ok=True)
    model_name = "QSPRModel_" + target
    model = SklearnModel(
        base_dir="output/final/models",
        # alg=KNeighbourssRegressor,
        alg=RandomForestRegressor,
        name=model_name
    )

    CrossValAssessor("r2")(model, dataset)
    TestSetAssessor("r2")(model, dataset)
    model.fitDataset(dataset)
    _ = model.save()

    # plot results
    path = '/home/s2861704/ppk/test_kin/output/final/'
    fullname = os.path.join(path, target)
    outdir = fullname + '.png'

    plot = CorrelationPlot([model])
    axes, summary = plot.make(save=True, show=True, out_path=outdir)

In [4]:
targets_kinase200 = df.columns[1:42].tolist()
display(targets_kinase200)
print(len(targets_kinase200))

selected_targets = targets_kinase200.copy()
del selected_targets[32:]
del selected_targets[29:31]
del selected_targets[16:18]
del selected_targets[11:14]
del selected_targets[8]
del selected_targets[4:7]
display(selected_targets)
print(len(selected_targets))

top_targets = targets_kinase200.copy()
del top_targets[32:]
del top_targets[29:31]
del top_targets[26]
del top_targets[22:25]
del top_targets[10:21]
del top_targets[8]
del top_targets[4:7]
del top_targets[0:2]
display(top_targets)
print(len(top_targets))

['O60674',
 'P00519',
 'P00533',
 'P04626',
 'P04629',
 'P06213',
 'P06239',
 'P07333',
 'P07949',
 'P08069',
 'P08581',
 'P09581',
 'P09619',
 'P10721',
 'P11362',
 'P12931',
 'P16234',
 'P17948',
 'P21802',
 'P22455',
 'P22607',
 'P23458',
 'P29317',
 'P29597',
 'P30530',
 'P35968',
 'P36888',
 'P43405',
 'P52333',
 'P54760',
 'Q02763',
 'Q05397',
 'Q06187',
 'Q06418',
 'Q07912',
 'Q08345',
 'Q08881',
 'Q12866',
 'Q16288',
 'Q16620',
 'Q16832']

41


['O60674',
 'P00519',
 'P00533',
 'P04626',
 'P07333',
 'P08069',
 'P08581',
 'P11362',
 'P12931',
 'P21802',
 'P22455',
 'P22607',
 'P23458',
 'P29317',
 'P29597',
 'P30530',
 'P35968',
 'P36888',
 'P43405',
 'P52333',
 'Q05397']

21


['P00533',
 'P04626',
 'P07333',
 'P08069',
 'P23458',
 'P35968',
 'P43405',
 'P52333',
 'Q05397']

9


In [5]:
test_mols = df['SMILES'].tolist()
display(test_mols)
print(len(test_mols))

['BrC(Cn1ncc2c(N3CCCC3)ncnc21)c1ccccc1',
 'BrC(Cn1ncc2c(N3CCCCC3)ncnc21)c1ccccc1',
 'BrC(Cn1ncc2c(NCc3ccccc3)ncnc21)c1ccccc1',
 'Brc1cc(CNc2ncc(Br)c(Nc3cc(C4CC4)[nH]n3)n2)on1',
 'Brc1cc2c(NCc3ccccc3)ncnc2s1',
 'Brc1cc2c(NCc3ccccn3)ncnc2s1',
 'Brc1cc2c(NCc3cccs3)ncnc2s1',
 'Brc1cc2c(NCc3ccncc3)ncnc2s1',
 'Brc1cc2c(Nc3ccccc3)ncnc2s1',
 'Brc1ccc(-c2cnc3nnc(Cc4c[nH]c5ccccc45)n3n2)cc1',
 'Brc1ccc(-c2cnc3nnc(Cc4ccc5ncccc5c4)n3n2)cc1',
 'Brc1ccc(-c2nc3cc(Nc4ccnc5ccccc45)ccc3[nH]2)cc1',
 'Brc1ccc(-c2nc3cc(Nc4ncnc5ccccc45)ccc3[nH]2)cc1',
 'Brc1ccc(-c2nnc(Cn3nnc4ccccc43)o2)cc1',
 'Brc1ccc(Br)c(Nc2ncc(Br)c(Nc3cc(C4CC4)[nH]n3)n2)c1',
 'Brc1ccc(C2=NN(c3ccccc3)C(c3ccc4ccccc4c3)C2)cc1',
 'Brc1ccc(C2=NN(c3ccccc3)C(c3cccc4ccccc34)C2)cc1',
 'Brc1ccc(C2=NN(c3nc(-c4ccc(Br)cc4)cs3)C(c3ccc4c(c3)OCCO4)C2)cc1',
 'Brc1ccc(C2=NN(c3nc(-c4ccc(Br)cc4)cs3)C(c3ccc4c(c3)OCO4)C2)cc1',
 'Brc1ccc(CSc2nnc(Cn3nnc4ccccc43)o2)cc1',
 'Brc1ccc(Cc2nnc(Cn3nnc4ccccc43)o2)cc1',
 'Brc1ccc(Nc2nc(N3CCOCC3)nc3c2ncn3C2CCCCO2)cc1',
 'B

37004


In [ ]:
for target in top_targets:
    print(target)
    data_loc = "/home/s2861704/ppk/test_kin/output/final/data/QSPRDataset_" + target + "/QSPRDataset_" + target + "_meta.json"
    model_loc = "/home/s2861704/ppk/test_kin/output/final/models/QSPRModel_" + target + "/QSPRModel_" + target + "_meta.json"
    dataset = QSPRDataset.fromFile(
        data_loc)
    model = SklearnModel.fromFile(
        model_loc)
    model.predictMols(test_mols)